In [1]:
import scipy.linalg as la
import scipy.sparse.linalg as spla
from scipy.sparse import lil_matrix
from scipy.sparse import csr_matrix
import scipy.sparse as sps
#import scipy.sparse.lil_matrix as ssplil
import scipy.io as sio
import numpy as np
import vibrationtesting as vt
import math
#from vibrationtesting import sparsematriceshandler

### Comparing results from system to sparsematriceshandler modules

**Issues observed:**

* sparse.eigsh does not spit all the eigen values and the eigen vectors. Apparently, the first eigen value and eigen vector was missing and rest of them are matching well. 

* Please note that, because of missing first eigen value and eigen vector using eigsh, I have manually entered those values as inputs while testing mode_expansion below. And both the modules results match well.



### Guyan Reduction and sos_modal

### 1. system module results

In [2]:
%matplotlib inline
mat_contents=sio.loadmat('FEmodaldata/WingBeamforMAC.mat')
K = (mat_contents['Kr'])
M = (mat_contents['Mr'])
Kbm = K.todense()
Mbm = M.todense()

omega, zeta, PsiBM = vt.sos_modal(Mbm, Kbm)
Psi_1 = PsiBM
Psi_1.shape

(75, 75)

In [3]:
omega=np.array([omega[0], omega[1], omega[2], omega[3], omega[4], omega[5]])
omega

array([ 140.92843994,  612.29446952, 1193.32651183, 1500.04377755,
       2542.04487083, 2802.33586918])

### 2. sparsematriceshandler module results

In [4]:
mat_contents=sio.loadmat('FEmodaldata/Wing_BeamFEModeldata.mat')
K = (mat_contents['K'])
M = (mat_contents['M'])

In [5]:
K

<141x141 sparse matrix of type '<class 'numpy.float64'>'
	with 1627 stored elements in Compressed Sparse Column format>

In [6]:
M

<141x141 sparse matrix of type '<class 'numpy.float64'>'
	with 1537 stored elements in Compressed Sparse Column format>

**WFEM Matlab, indices**

In [7]:
master = np.array([[ 2,  3,  4,  5,  6,  8,  9, 10, 11, 12, 14, 15, 16, 17, 18, 20,
        21, 22, 23, 24, 26, 27, 28, 29, 30, 32, 33, 34, 35, 36, 38, 39,
        40, 41, 42, 44, 45, 46, 47, 48, 50, 51, 52, 53, 54, 56, 57, 58,
        59, 60, 62, 63, 64, 65, 66, 68, 69, 70, 71, 72, 74, 75, 76, 77,
        78, 80, 81, 82, 83, 84, 86, 87, 88, 89, 90]])
master.shape

(1, 75)

In [8]:
[Mred, Kred, T, master, slave] = vt.guyan_forsparse(M, K, master=master, fraction=None)

/home/sainag/.local/lib/python3.6/site-packages/scipy-1.3.0.dev0+5875fd3-py3.6-linux-x86_64.egg/scipy/sparse/linalg/dsolve/linsolve.py:133: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)


In [9]:
Mred

<75x75 sparse matrix of type '<class 'numpy.float64'>'
	with 1023 stored elements in Compressed Sparse Row format>

In [10]:
Kred

<75x75 sparse matrix of type '<class 'numpy.float64'>'
	with 1007 stored elements in Compressed Sparse Row format>

In [11]:
[omega,zeta,Psi] = vt.sos_modal_forsparse(Mred,Kred)
Psi.shape

(75, 75)

In [12]:
omega=np.array([omega[0], omega[1], omega[2], omega[3], omega[4], omega[5]])
omega

array([[ 140.92843993],
       [ 612.29446952],
       [1193.32651183],
       [1500.04377756],
       [2542.04487083],
       [2802.33586918]])

### mode_expansion comparison between both the modules

In [13]:
mat_contents=sio.loadmat('Modeshapedata/1stNaturalFreq15.mat')
U1 = mat_contents['U1']
mat_contents=sio.loadmat('Modeshapedata/2ndNaturalFreq15.mat')
U2 = mat_contents['U2']
mat_contents=sio.loadmat('Modeshapedata/3rdNaturalFreq15.mat')
U3 = mat_contents['U3']
mat_contents=sio.loadmat('Modeshapedata/4rthNaturalFreq15.mat')
U4 = mat_contents['U4']
mat_contents=sio.loadmat('Modeshapedata/5thNaturalFreq15.mat')
U5 = mat_contents['U5']
mat_contents=sio.loadmat('Modeshapedata/6thNaturalFreq15.mat')
U6 = mat_contents['U6']
Psi_1 = np.array(U1)
Psi_2 = np.array(U2)
Psi_3 = np.array(U3)
Psi_4 = np.array(U4)
Psi_5 = np.array(U5)
Psi_6 = np.array(U6)
Psi_1=np.column_stack((Psi_1,Psi_2,Psi_3,Psi_4,Psi_5,Psi_6))
Psi_abs = np.abs(Psi_1)*np.real(np.sign(Psi_1))
Psi = Psi_abs
Psi.shape

(15, 6)

In [14]:
mat_contents=sio.loadmat('FEmodaldata/WingBeamforMAC.mat')
K = (mat_contents['Kr'])
M = (mat_contents['Mr'])

Kbm = K.todense()
Mbm = M.todense()
omega1, zeta, Psi1 = vt.sos_modal(Mbm, Kbm, damp_diag=0)
omega, zeta, PsiBM = vt.sos_modal_forsparse(M, K, damp_diag=0)
measured = np.array([[1,6,11,16,21,26,31,36,41,46,51,56,61,66,71]])
omega=np.array([140.92843994, omega[0], omega[1], omega[2], omega[3], omega[4]])
Psi_Test=vt.mode_expansion_from_model(Psi, omega, Mbm, Kbm, measured)
Psi_Testsparse=vt.mode_expansion_from_model_forsparse(Psi, omega, M, K, measured)

**Expanded mode shapes with Dense or Full matrices as inputs:**

In [15]:
Psi_Test

array([[ 2.34856175e-18, -1.29474969e-15, -8.23521093e-16,
         1.79406168e-13,  1.40275019e-14,  1.82591563e-13],
       [ 5.24170771e+00, -2.71276151e+01,  5.28434535e+01,
         1.07210143e+02,  7.11390950e+01, -2.69817343e+02],
       [-5.69631727e-03,  3.82621917e-02, -3.21204866e-01,
         4.39671521e-02, -6.87923759e-01, -2.56166647e-01],
       [ 7.20892485e-03,  3.42861348e-03,  7.80711833e-03,
         1.49246664e+00, -1.77769363e-02, -2.67358697e+00],
       [-4.61728261e-18,  4.19901244e-17, -1.46690067e-18,
        -2.00508184e-15, -3.45084080e-17,  6.20138633e-15],
       [-1.71700959e-17, -1.18352703e-15, -6.06902158e-16,
         1.80886602e-13,  1.49201290e-14,  1.94406257e-13],
       [ 4.68720936e+00, -2.69457382e+01,  5.21831863e+01,
        -7.83341953e+00,  7.14115345e+01, -4.06255384e+01],
       [-3.31704994e-03,  3.52716226e-02, -3.20799881e-01,
         8.99060729e-02, -6.75300587e-01, -1.05297526e-01],
       [ 5.49411743e-03, -1.26275162e-02,  7.859

**Expanded mode shapes with Sparse matrices as inputs:**

In [16]:
Psi_Testsparse

array([[ 2.34856175e-18, -1.29474969e-15, -8.23521093e-16,
         1.79406168e-13,  1.40275019e-14,  1.82591563e-13],
       [ 5.24170771e+00, -2.71276151e+01,  5.28434535e+01,
         1.07210143e+02,  7.11390950e+01, -2.69817343e+02],
       [-5.69631727e-03,  3.82621917e-02, -3.21204866e-01,
         4.39671521e-02, -6.87923759e-01, -2.56166647e-01],
       [ 7.20892485e-03,  3.42861348e-03,  7.80711833e-03,
         1.49246664e+00, -1.77769363e-02, -2.67358697e+00],
       [-4.61728261e-18,  4.19901244e-17, -1.46690067e-18,
        -2.00508184e-15, -3.45084080e-17,  6.20138633e-15],
       [-1.71700959e-17, -1.18352703e-15, -6.06902158e-16,
         1.80886602e-13,  1.49201290e-14,  1.94406257e-13],
       [ 4.68720936e+00, -2.69457382e+01,  5.21831863e+01,
        -7.83341953e+00,  7.14115345e+01, -4.06255384e+01],
       [-3.31704994e-03,  3.52716226e-02, -3.20799881e-01,
         8.99060729e-02, -6.75300587e-01, -1.05297526e-01],
       [ 5.49411743e-03, -1.26275162e-02,  7.859